# KOI Data Preprocessing - Remove Unwanted Columns

This notebook preprocesses the KOI (Kepler Objects of Interest) dataset by removing columns that should not be used for machine learning analysis:
- `kepid`: Identifier column (not predictive)
- `koi_datalink_dvr`: Link to DV report (not predictive)

## Purpose:
- Load the original KOI dataset
- Remove identifier and link columns
- Save the cleaned dataset for analysis
- Provide summary statistics

## 1. Import Required Libraries

In [33]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os

# Set display options for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Load Original Dataset

In [34]:
# Load the original KOI dataset
print("Loading KOI Selected Data.csv...")

# Read the CSV file, skipping the comment lines that start with '#'
df_original = pd.read_csv('TESS Selected Data.csv', comment='#')

print(f"Original dataset shape: {df_original.shape}")
print(f"Number of rows: {df_original.shape[0]:,}")
print(f"Number of columns: {df_original.shape[1]}")

print("\nColumn names in original dataset:")
for i, col in enumerate(df_original.columns, 1):
    print(f"{i:2d}. {col}")

print("\nFirst few rows of original data:")
df_original.head()

Loading KOI Selected Data.csv...
Original dataset shape: (7703, 14)
Number of rows: 7,703
Number of columns: 14

Column names in original dataset:
 1. tid
 2. tfopwg_disp
 3. pl_tranmid
 4. pl_orbper
 5. pl_trandurh
 6. pl_trandep
 7. pl_rade
 8. pl_insol
 9. pl_eqt
10. st_tmag
11. st_dist
12. st_teff
13. st_logg
14. st_rad

First few rows of original data:


,tid,tfopwg_disp,pl_tranmid,pl_orbper,pl_trandurh,pl_trandep,pl_rade,pl_insol,pl_eqt,st_tmag,st_dist,st_teff,st_logg,st_rad
0,50365310,FP,2.459230e+06,2.171348,2.01722,656.886099,5.818163,22601.948581,3127.204052,9.604000,485.735,10249.0,4.19,2.16986
1,88863718,PC,2.459988e+06,1.931646,3.16600,1286.000000,11.215400,44464.500000,4045.000000,9.423440,295.862,7070.0,4.03,2.01000
2,124709665,FP,2.459225e+06,1.867557,1.40800,1500.000000,23.752900,2860.610000,2037.000000,9.299501,943.109,8924.0,NaN,5.73000
3,106997505,FP,2.458493e+06,2.743230,3.16700,383.410000,NaN,1177.360000,1631.000000,9.300300,7728.170,5388.5,4.15,NaN
4,238597883,FP,2.459987e+06,3.573014,3.37000,755.000000,11.311300,54679.300000,4260.000000,9.135500,356.437,9219.0,4.14,2.15000


## 3. Identify and Remove Unwanted Columns

In [35]:
# Define columns to remove for machine learning analysis
columns_to_remove = ['tid']

print("Columns to be removed:")
for col in columns_to_remove:
    if col in df_original.columns:
        print(f"✓ {col} - Found in dataset")
    else:
        print(f"✗ {col} - NOT found in dataset")

# Check if the columns exist before removing
existing_columns_to_remove = [col for col in columns_to_remove if col in df_original.columns]

print(f"\nColumns that will actually be removed: {existing_columns_to_remove}")

# Remove the unwanted columns
df_cleaned = df_original.drop(columns=existing_columns_to_remove, errors='ignore')

print(f"\nDataset shape after removing columns:")
print(f"Original: {df_original.shape}")
print(f"Cleaned:  {df_cleaned.shape}")
print(f"Columns removed: {df_original.shape[1] - df_cleaned.shape[1]}")

print(f"\nRemaining columns: {df_cleaned.shape[1]}")
for i, col in enumerate(df_cleaned.columns, 1):
    print(f"{i:2d}. {col}")

Columns to be removed:
✓ tid - Found in dataset

Columns that will actually be removed: ['tid']

Dataset shape after removing columns:
Original: (7703, 14)
Cleaned:  (7703, 13)
Columns removed: 1

Remaining columns: 13
 1. tfopwg_disp
 2. pl_tranmid
 3. pl_orbper
 4. pl_trandurh
 5. pl_trandep
 6. pl_rade
 7. pl_insol
 8. pl_eqt
 9. st_tmag
10. st_dist
11. st_teff
12. st_logg
13. st_rad


## 4. Data Quality Check

In [36]:
# Perform basic data quality checks on the cleaned dataset
print("=== DATA QUALITY SUMMARY ===\n")

# Basic info
print(f"Dataset shape: {df_cleaned.shape}")
print(f"Memory usage: {df_cleaned.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Data types
print(f"\nData types:")
print(df_cleaned.dtypes.value_counts())

# Missing values
print(f"\nMissing values summary:")
missing_count = df_cleaned.isnull().sum()
missing_percent = (missing_count / len(df_cleaned) * 100).round(2)
missing_summary = pd.DataFrame({
    'Column': missing_count.index,
    'Missing_Count': missing_count.values,
    'Missing_Percent': missing_percent.values
}).sort_values('Missing_Count', ascending=False)

# Show only columns with missing values
columns_with_missing = missing_summary[missing_summary['Missing_Count'] > 0]
if len(columns_with_missing) > 0:
    print(f"Columns with missing values: {len(columns_with_missing)}")
    print(columns_with_missing)
else:
    print("No missing values found!")

# Target variable distribution
if 'koi_disposition' in df_cleaned.columns:
    print(f"\nTarget variable (koi_disposition) distribution:")
    disposition_counts = df_cleaned['koi_disposition'].value_counts()
    print(disposition_counts)
    print(f"\nPercentages:")
    print((disposition_counts / len(df_cleaned) * 100).round(2))

print(f"\nCleaned dataset preview:")
df_cleaned.head()

=== DATA QUALITY SUMMARY ===

Dataset shape: (7703, 13)
Memory usage: 1.08 MB

Data types:
float64    12
object      1
Name: count, dtype: int64

Missing values summary:
Columns with missing values: 8
       Column  Missing_Count  Missing_Percent
11    st_logg            856            11.11
12     st_rad            507             6.58
5     pl_rade            506             6.57
7      pl_eqt            311             4.04
9     st_dist            215             2.79
6    pl_insol            176             2.28
10    st_teff            161             2.09
2   pl_orbper            107             1.39

Cleaned dataset preview:


,tfopwg_disp,pl_tranmid,pl_orbper,pl_trandurh,pl_trandep,pl_rade,pl_insol,pl_eqt,st_tmag,st_dist,st_teff,st_logg,st_rad
0,FP,2.459230e+06,2.171348,2.01722,656.886099,5.818163,22601.948581,3127.204052,9.604000,485.735,10249.0,4.19,2.16986
1,PC,2.459988e+06,1.931646,3.16600,1286.000000,11.215400,44464.500000,4045.000000,9.423440,295.862,7070.0,4.03,2.01000
2,FP,2.459225e+06,1.867557,1.40800,1500.000000,23.752900,2860.610000,2037.000000,9.299501,943.109,8924.0,NaN,5.73000
3,FP,2.458493e+06,2.743230,3.16700,383.410000,NaN,1177.360000,1631.000000,9.300300,7728.170,5388.5,4.15,NaN
4,FP,2.459987e+06,3.573014,3.37000,755.000000,11.311300,54679.300000,4260.000000,9.135500,356.437,9219.0,4.14,2.15000


In [37]:
# Calculate missing percentage for each column
missing_percent = df_cleaned.isnull().sum() / len(df_cleaned) * 100

# Find columns with more than 75% missing data
high_missing_cols = missing_percent[missing_percent > 30].index.tolist()

print("Columns with >30% missing data that will be dropped:")
for col in high_missing_cols:
    print(f"- {col}: {missing_percent[col]:.2f}% missing")

# Create new dataframe without high-missing columns
df_cleaned_reduced = df_cleaned.drop(columns=high_missing_cols)

print(f"\nOriginal shape: {df_cleaned.shape}")
print(f"New shape: {df_cleaned_reduced.shape}")
print(f"Columns removed: {len(high_missing_cols)}")

# Update the main dataframe
df_cleaned = df_cleaned_reduced

Columns with >30% missing data that will be dropped:

Original shape: (7703, 13)
New shape: (7703, 13)
Columns removed: 0


## 5. Handle Missing Values (Non-Destructive)

In [38]:
# Handle missing values using non-destructive methods
print("=== HANDLING MISSING VALUES ===\n")

# Create a copy for processing
df_processed = df_cleaned.copy()

# Check current missing values
initial_missing = df_processed.isnull().sum().sum()
print(f"Total missing values before processing: {initial_missing:,}")

# Separate numerical and categorical columns
numerical_cols = df_processed.select_dtypes(include=[np.number]).columns
categorical_cols = df_processed.select_dtypes(include=['object']).columns

print(f"\nNumerical columns to process: {len(numerical_cols)}")
print(f"Categorical columns to process: {len(categorical_cols)}")

# Strategy 1: Fill numerical missing values with median
for col in numerical_cols:
    if df_processed[col].isnull().sum() > 0:
        median_val = df_processed[col].median()
        df_processed[col].fillna(median_val, inplace=True)
        print(f"✓ {col}: filled with median {median_val:.2f}")

# Strategy 2: Fill categorical missing values with mode
for col in categorical_cols:
    if df_processed[col].isnull().sum() > 0:
        mode_val = df_processed[col].mode()[0]
        df_processed[col].fillna(mode_val, inplace=True)
        print(f"✓ {col}: filled with mode '{mode_val}'")

# Special handling for astronomical data
special_fills = {
    'pl_orbeccen': 0.0,  # Assuming circular orbit
    'pl_imppar': 0.5,    # Assuming average impact parameter
}

for col, fill_val in special_fills.items():
    if col in df_processed.columns and df_processed[col].isnull().sum() > 0:
        df_processed[col].fillna(fill_val, inplace=True)
        print(f"✓ {col}: filled with domain-specific value {fill_val}")

# Final check
final_missing = df_processed.isnull().sum().sum()
print(f"\nTotal missing values after processing: {final_missing:,}")
print(f"Missing values filled: {initial_missing - final_missing:,}")

# Create a summary of the changes
print("\nSummary of changes:")
for col in df_processed.columns:
    initial = df_cleaned[col].isnull().sum()
    final = df_processed[col].isnull().sum()
    if initial > 0:
        print(f"{col}: {initial:,} → {final:,} missing values")

# Store processed dataframe
df_cleaned = df_processed.copy()

=== HANDLING MISSING VALUES ===

Total missing values before processing: 2,839

Numerical columns to process: 12
Categorical columns to process: 1
✓ pl_orbper: filled with median 4.09
✓ pl_rade: filled with median 10.54
✓ pl_insol: filled with median 363.90
✓ pl_eqt: filled with median 1183.01
✓ st_dist: filled with median 365.01
✓ st_teff: filled with median 5800.55
✓ st_logg: filled with median 4.33
✓ st_rad: filled with median 1.23

Total missing values after processing: 0
Missing values filled: 2,839

Summary of changes:
pl_orbper: 107 → 0 missing values
pl_rade: 506 → 0 missing values
pl_insol: 176 → 0 missing values
pl_eqt: 311 → 0 missing values
st_dist: 215 → 0 missing values
st_teff: 161 → 0 missing values
st_logg: 856 → 0 missing values
st_rad: 507 → 0 missing values


C:\Users\navid\AppData\Local\Temp\ipykernel_11460\3283865381.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed[col].fillna(median_val, inplace=True)
C:\Users\navid\AppData\Local\Temp\ipykernel_11460\3283865381.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For

In [39]:
# Save the cleaned dataset
output_filename = 'TESS_Cleaned_Data.csv'

print(f"Saving cleaned dataset to: {output_filename}")

# Save without index to avoid adding an extra column
df_cleaned.to_csv(output_filename, index=False)

# Verify the saved file
if os.path.exists(output_filename):
    file_size = os.path.getsize(output_filename) / 1024**2  # Size in MB
    print(f"✓ File saved successfully!")
    print(f"  File size: {file_size:.2f} MB")
    
    # Quick verification by reading back
    df_verify = pd.read_csv(output_filename)
    print(f"  Verification - Shape: {df_verify.shape}")
    print(f"  Verification - Columns match: {list(df_verify.columns) == list(df_cleaned.columns)}")
else:
    print("✗ Error: File was not saved!")

print(f"\n=== PREPROCESSING COMPLETE ===")
print(f"Original dataset: {df_original.shape[0]:,} rows × {df_original.shape[1]} columns")
print(f"Cleaned dataset:  {df_cleaned.shape[0]:,} rows × {df_cleaned.shape[1]} columns")
print(f"Columns removed:  {existing_columns_to_remove}")
print(f"Output file:      {output_filename}")

print(f"\nThe cleaned dataset is ready for machine learning analysis!")

Saving cleaned dataset to: TESS_Cleaned_Data.csv
✓ File saved successfully!
  File size: 0.78 MB
  Verification - Shape: (7703, 13)
  Verification - Columns match: True

=== PREPROCESSING COMPLETE ===
Original dataset: 7,703 rows × 14 columns
Cleaned dataset:  7,703 rows × 13 columns
Columns removed:  ['tid']
Output file:      TESS_Cleaned_Data.csv

The cleaned dataset is ready for machine learning analysis!
✓ File saved successfully!
  File size: 0.78 MB
  Verification - Shape: (7703, 13)
  Verification - Columns match: True

=== PREPROCESSING COMPLETE ===
Original dataset: 7,703 rows × 14 columns
Cleaned dataset:  7,703 rows × 13 columns
Columns removed:  ['tid']
Output file:      TESS_Cleaned_Data.csv

The cleaned dataset is ready for machine learning analysis!


## Summary

This notebook successfully:

1. **Loaded** the original KOI Selected Data.csv file
2. **Identified** and removed unwanted columns:
   - `kepid` (identifier, not predictive)
   - `koi_datalink_dvr` (link to DV report, not predictive)
3. **Performed** data quality checks on the cleaned dataset
4. **Saved** the cleaned dataset as `KOI_Cleaned_Data.csv`

### Next Steps:
- Use the cleaned dataset (`KOI_Cleaned_Data.csv`) for machine learning analysis
- The dataset is now ready for feature importance analysis and classification tasks
- All identifier and link columns have been removed to prevent data leakage

### Files Created:
- `KOI_Cleaned_Data.csv` - Cleaned dataset ready for ML analysis